In [125]:
import pandas as pd
import plotly.graph_objects as go
import math
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from pandas_datareader import data as pdr
import yfinance as yfin


In [126]:
df = pd.read_csv('TSLA.csv')
df = df.set_index('Date')
df = df.dropna()
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-10-22,14.104000,14.383333,13.960000,14.114667,14.114667,42378000
2015-10-23,14.333333,14.356667,13.846000,13.939333,13.939333,63532500
2015-10-26,14.092000,14.392000,14.000000,14.350667,14.350667,50871000
2015-10-27,14.322667,14.473333,13.834000,14.023333,14.023333,52791000
2015-10-28,14.087333,14.230000,13.886667,14.197333,14.197333,40929000
...,...,...,...,...,...,...
2023-04-13,182.960007,186.500000,180.940002,185.899994,185.899994,112933000
2023-04-14,183.949997,186.279999,182.009995,185.000000,185.000000,96306500
2023-04-17,186.320007,189.690002,182.690002,187.039993,187.039993,116662200


In [127]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.High,
                    mode='lines',
                    name='High',
                    marker_color = '#2CA02C',
                    #visible = "legendonly"
                         ))
fig.add_trace(go.Scatter(x = df.index, y = df.Low,
                    mode='lines',
                    name='Low',
                    marker_color = '#D62728',
                    #visible = "legendonly"
                         ))
fig.add_trace(go.Scatter(x = df.index, y = df.Open,
                    mode='lines',
                    name='Open',
                    marker_color = '#FF7F0E',
                    #visible = "legendonly"
                         ))
fig.add_trace(go.Scatter(x = df.index, y = df.Close,
                    mode='lines',
                    name='Close',
                    marker_color = '#1F77B4'))

fig.update_layout(
    title='Closing price history',
    titlefont_size = 28,
    
    xaxis = dict(
        title='Date',
        titlefont_size=16,
        tickfont_size=14),
    
    height = 800,
    
    yaxis=dict(
        title='Price',
        titlefont_size=16,
        tickfont_size=14),
    
    legend=dict(
        y=0,
        x=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))

fig.show()

In [128]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .8)
training_data_len

1508

In [129]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

array([[0.01133056],
       [0.01089266],
       [0.01191999],
       ...,
       [0.44322063],
       [0.43640232],
       [0.42711142]])

In [130]:
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i <= 61:
        print(x_train)
        print(y_train)
        print()

[array([0.01133056, 0.01089266, 0.01191999, 0.01110245, 0.01153703,
       0.01131558, 0.01053301, 0.01167523, 0.01076945, 0.01464565,
       0.01466896, 0.0147672 , 0.01359668, 0.01212645, 0.01255603,
       0.0115337 , 0.0105763 , 0.01176181, 0.01171019, 0.01288737,
       0.01300892, 0.01271088, 0.01233458, 0.01241783, 0.0143143 ,
       0.01464232, 0.01441754, 0.01557141, 0.01470559, 0.01482547,
       0.01443752, 0.0145624 , 0.01382812, 0.01346181, 0.01388639,
       0.01221303, 0.01247278, 0.0128907 , 0.01512518, 0.01493869,
       0.01445084, 0.0148005 , 0.01436592, 0.01432429, 0.01446915,
       0.01419942, 0.01557141, 0.01572126, 0.01604095, 0.01327699,
       0.01328032, 0.01254937, 0.01198492, 0.01121068, 0.0106862 ,
       0.01103918, 0.00943076, 0.01040813, 0.01020999, 0.01016504])]
[0.009162688042811326]

[array([0.01133056, 0.01089266, 0.01191999, 0.01110245, 0.01153703,
       0.01131558, 0.01053301, 0.01167523, 0.01076945, 0.01464565,
       0.01466896, 0.0147672 , 0.0

In [131]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

(1448, 60, 1)

In [132]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, batch_size = 1, epochs = 1)

1448/1448 [==============================] - 42s 25ms/step - loss: 9.7019e-04


In [133]:
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range (60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

rsme = np.sqrt(np.mean(predictions - y_test) ** 2)
rsme

12/12 [==============================] - 2s 20ms/step


5.686811961266476

In [134]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_4848\3011436284.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = train.index, y = train.Close,
                    mode='lines',
                    name='Close',
                    marker_color = '#1F77B4'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Close,
                    mode='lines',
                    name='Val',
                    marker_color = '#FF7F0E'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Predictions,
                    mode='lines',
                    name='Predictions',
                    marker_color = '#2CA02C'))

fig.update_layout(
    title='Model',
    titlefont_size = 28,
    hovermode = 'x',
    xaxis = dict(
        title='Date',
        titlefont_size=16,
        tickfont_size=14),
    
    height = 800,
    
    yaxis=dict(
        title='Close price',
        titlefont_size=16,
        tickfont_size=14),
    legend=dict(
        y=0,
        x=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))

fig.show()

In [136]:
valid

,Close,Predictions
Date,,
2021-10-19,288.089996,279.170380
2021-10-20,288.600006,282.892548
2021-10-21,298.000000,286.228455
2021-10-22,303.226654,290.055450
2021-10-25,341.619995,294.246490
...,...,...
2023-04-13,185.899994,199.301025
2023-04-14,185.000000,197.953888
2023-04-17,187.039993,196.980148


In [137]:
last_60_days = data[-60:].values
last_60_days_scaled = scaler.transform(last_60_days)
X_test = []
X_test.append(last_60_days_scaled)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1], 1))
pred_price = model.predict(X_test)
pred_price = scaler.inverse_transform(pred_price)
print ('Predicting the closing price for the next day')
print()
print(pred_price)

1/1 [==============================] - 0s 124ms/step
Predicting the closing price for the next day

[[195.4678]]


In [138]:
yfin.pdr_override()
spy = pdr.get_data_yahoo('TSLA', start = '2023-04-21', end = '2023-04-22')
print(spy)

[*********************100%***********************]  1 of 1 completed
                  Open   High         Low       Close   Adj Close     Volume
Date                                                                        
2023-04-21  164.800003  166.0  161.320007  165.080002  165.080002  123539000
